#Exploratory Data Analysis — Kat Hanner — June 15, 2020

In [156]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sqlite3

In [157]:
df_bomGross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
print('BOM Movie Gross Shape: ', df_bomGross.shape)
print('BOM Movie Gross Info: ', df_bomGross.info())
df_bomGross.head()

BOM Movie Gross Shape:  (3387, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
BOM Movie Gross Info:  None


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [159]:
#convert tsv to csv

tsv_file = './zippedData/rt.movie_info.tsv.gz'
convert = pd.read_table(tsv_file, delimiter='\t')
convert.to_csv('./zippedData/rt.movie_info.csv.gz')
newdf = pd.read_csv('./zippedData/rt.movie_info.csv.gz')
newdf.info()

#coding a loop: try later
# for file in datafiles:
    # d_tsv[file] = pd.read_csv(file, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    1560 non-null   int64 
 1   id            1560 non-null   int64 
 2   synopsis      1498 non-null   object
 3   rating        1557 non-null   object
 4   genre         1552 non-null   object
 5   director      1361 non-null   object
 6   writer        1111 non-null   object
 7   theater_date  1201 non-null   object
 8   dvd_date      1201 non-null   object
 9   currency      340 non-null    object
 10  box_office    340 non-null    object
 11  runtime       1530 non-null   object
 12  studio        494 non-null    object
dtypes: int64(2), object(11)
memory usage: 158.6+ KB


In [160]:
#gives error UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 4: invalid start byte

tsv_file_r = './zippedData/rt.reviews.tsv.gz'
convert_r = pd.read_table(tsv_file_r, delimiter='\t')
convert_r.to_csv('./zippedData/rt.reviews.csv.gz')
newdf_r = pd.read_csv('./zippedData/rt.reviews.csv.gz', engine='python')
newdf_r.info()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 4: invalid start byte

In [161]:
from glob import glob #finds all pathnames
datafiles = glob('./zippedData/*.csv.gz')
datafiles

['./zippedData/imdb.title.crew.csv.gz',
 './zippedData/rt.movie_info.csv.gz',
 './zippedData/tmdb.movies.csv.gz',
 './zippedData/imdb.title.akas.csv.gz',
 './zippedData/imdb.name.basics.csv.gz',
 './zippedData/imdb.title.basics.csv.gz',
 './zippedData/tn.movie_budgets.csv.gz',
 './zippedData/bom.movie_gross.csv.gz',
 './zippedData/imdb.title.principals.csv.gz']

In [162]:
datafiles_tsv = glob('./zippedData/*.tsv.gz')
datafiles_tsv

['./zippedData/rt.reviews.tsv.gz', './zippedData/rt.movie_info.tsv.gz']

In [225]:
type(datafiles)
d = {} #creates dictionary 
## d_tsv = {} #tried to create dictionary for tsv files; preference for just converting all files to the same file type

In [226]:
for file in datafiles:
    d[file] = pd.read_csv(file)
    ## d_tsv[file] = pd.read_table(file, delimiter='\t') #tried to loop through tsv dictionary; fail

In [228]:
d.keys()
d_tsv.keys() #d_tsv dictionary failed

dict_keys(['./zippedData/imdb.title.crew.csv.gz', './zippedData/rt.movie_info.csv.gz', './zippedData/tmdb.movies.csv.gz', './zippedData/imdb.title.akas.csv.gz', './zippedData/imdb.name.basics.csv.gz', './zippedData/imdb.title.basics.csv.gz', './zippedData/tn.movie_budgets.csv.gz', './zippedData/bom.movie_gross.csv.gz', './zippedData/imdb.title.principals.csv.gz'])

In [167]:
type(datafiles)

list

In [171]:
datafiles_dict = {}
for filename in datafiles:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".gz", "").replace(".", "_") # cleaning the filenames
    filename_df = pd.read_csv(filename, index_col=0)
    datafiles_dict[filename_cleaned] = filename_df

In [178]:
datafiles_dict.keys()

dict_keys(['imdb_title_crew', 'rt_movie_info', 'tmdb_movies', 'imdb_title_akas', 'imdb_name_basics', 'imdb_title_basics', 'tn_movie_budgets', 'bom_movie_gross', 'imdb_title_principals'])

In [173]:
datafiles_dict['imdb_title_crew'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0285252 to tt9010172
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   directors  140417 non-null  object
 1   writers    110261 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [203]:
conn = sqlite3.connect("movies_db.sqlite")

In [211]:
def create_sql_table_from_df(df, name, conn):
    try:
        df.to_sql(name, conn)
        print(f"Created table {name}")
    
    except Exception as e:
        print(f"could not make table {name}")
        print(e)

In [212]:
for name, table in datafiles_dict.items():
    create_sql_table_from_df(table, name, conn)

could not make table imdb_title_crew
Table 'imdb_title_crew' already exists.
could not make table rt_movie_info
Table 'rt_movie_info' already exists.
could not make table tmdb_movies
Table 'tmdb_movies' already exists.
could not make table imdb_title_akas
Table 'imdb_title_akas' already exists.
could not make table imdb_name_basics
Table 'imdb_name_basics' already exists.
could not make table imdb_title_basics
Table 'imdb_title_basics' already exists.
could not make table tn_movie_budgets
Table 'tn_movie_budgets' already exists.
could not make table bom_movie_gross
Table 'bom_movie_gross' already exists.
could not make table imdb_title_principals
Table 'imdb_title_principals' already exists.


In [213]:
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('imdb_title_crew',),
 ('tmdb_movies',),
 ('imdb_title_akas',),
 ('imdb_title_ratings',),
 ('imdb_name_basics',),
 ('imdb_title_basics',),
 ('tn_movie_budgets',),
 ('bom_movie_gross',),
 ('imdb_title_principals',),
 ('rt_movie_info',)]

In [214]:
for item in datafiles_dict.items():
        print("===============New DF===============")
        print(item[0])
        print(item[1].info())
        print("================End DF===============")

===============New DF===============
imdb_title_crew
<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0285252 to tt9010172
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   directors  140417 non-null  object
 1   writers    110261 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB
None
================End DF===============
===============New DF===============
rt_movie_info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   curre

In [208]:
cur = conn.cursor()

In [209]:
cur.execute("""
SELECT *
FROM tn_movie_budgets
ORDER BY release_date DESC; """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,50,"Sep 9, 2016",Sully,"$60,000,000","$125,070,033","$238,524,556"
1,30,"Sep 9, 2016",The Disappointments Room,"$15,000,000","$2,423,467","$3,503,761"
2,88,"Sep 9, 2016",Robinson Crusoe,"$13,000,000","$8,005,586","$42,446,455"
3,85,"Sep 9, 2016",When the Bough Breaks,"$10,000,000","$29,747,603","$30,768,449"
4,13,"Sep 9, 2016",Kicks,"$10,000,000","$150,191","$150,191"
...,...,...,...,...,...,...
5777,9,"Apr 1, 2005",Sin City,"$40,000,000","$74,103,820","$158,527,918"
5778,70,"Apr 1, 1996",Bang,"$10,000",$527,$527
5779,61,"Apr 1, 1988","Bright Lights, Big City","$25,000,000","$16,118,077","$16,118,077"
5780,21,"Apr 1, 1986",My Beautiful Laundrette,"$400,000",$0,$0
